# Chapter 09: Creating Command-line Interfaces

##### 運行前先確定當前工作目錄下是否有unitconverter資料夾和setup.py檔案

In [ ]:
!ls 

## 1. 測試unitconverter的基本功能

##### 將200 wh 的能量分別轉換為 j, hph的形式

In [ ]:
from unitconverter.Converter import get_table, convert_units
get_table('time').get_units()
convert_units('energy', 200, 'wh', ['j', 'hph'])

## 2. 撰寫unitconverter的Command-line Interface程式

##### File name: CLI.py

In [ ]:
# import the required libraries
import argparse
import inspect
from unitconverter.Converter import get_table, convert_units

In [ ]:
def run_cli():
    # create an ArgumentParser object, to which we can add arguments that are to be parsed from the command line
    parser = argparse.ArgumentParser(description='Tool for converting units')

    # The first argument will be used to define the conversion table that will be used in the unit conversion
    parser.add_argument(
        'table',
        metavar='TABLE',
        action='store',
        type=str,
        help='Unit table to use in conversion'
    )

    subparsers = parser.add_subparsers(help='operation to be performed')  # create subparser object

    list_table_parser = subparsers.add_parser('list')  # add subparser "list_table_parser" to deal with unit list command
    list_table_parser.set_defaults(which='list')  # set the default values for subparser "list_table_parser"

    # optional argument for the subparser for the mode to list all the possible conversions
    list_table_parser.add_argument(
        '-m', '--method',
        action='store_true',
        help='Also output the conversion method from the base unit'
    )

    conversion_parser = subparsers.add_parser('convert')  # add subparser "list_table_parser" to deal with unit conversion command
    conversion_parser.set_defaults(which='convert')  # set the default values for subparser "list_table_parser"

    # add positional requirements to the subparser for unit conversion mode
    conversion_parser.add_argument(
        'value',
        metavar='VALUE',
        type=float,
        action='store',
        help='The value to convert'
    )

    conversion_parser.add_argument(
        'from_unit',
        metavar='FROM',
        action='store',
        type=str,
        help='Unit to convert from'
    )

    conversion_parser.add_argument(
        'to_units',
        metavar='TO',
        action='store',
        nargs='+',
        type=str,
        help='Unit(s) to convert to'
    )

    props = parser.parse_args()  # call the parse_args function to parse the options from the command line

    # run either the _run_unit_list or _run_conversion function to perform the processing of the application
    if props.which == 'list':
        _run_unit_list(props)
    elif props.which == 'convert':
        _run_conversion(props)

In [ ]:
def _run_unit_list(props):
    """
    Runs the command line interface for unit listing mode.

    @param props Properties parsed by argparse
    """

    table = get_table(props.table)
    print 'Unit table %s can convert between the units:'
    for unit in table.get_units():
        if props.method:  # if argument "--method" is passed in
            if unit == table.base_unit:  # if unit is base unit, then don't show conversion formula
                formula = 'base unit'
            else:  # retrieve conversion formula 
                conversion = inspect.getsource(table.from_base_unit[unit])
                formula = conversion[conversion.index(':')+1:conversion.index('\n')].strip()
            print '%s (%s)' % (unit, formula)
        else:  # if argument "--method" does not passed in, just print out available units
            print unit

In [ ]:
def _run_conversion(props):
    """
    Runs the command line interface for unit conversion mode.

    @param props Properties parsed by argparse
    """

    # compute conversions
    results = convert_units(table_name=props.table,
                            value=props.value,
                            value_unit=props.from_unit,
                            targets=props.to_units)

    # print out conversion results
    for result in results:
        print '%f %s = %f %s' % (props.value, props.from_unit,
                                 result['converted_value'],
                                 result['dest_unit'])

## 3. 運行Command-line Interface環境

##### 先使用"python setup.py install"，安裝完以後方可以啟用命令行工具

In [ ]:
! python setup.py install

##### 顯示可以進行轉換的單位名稱

In [1]:
! unitconvert energy list

Unit table %s can convert between the units:
btu
j
wh
hph
cal
ev
kcal


##### 顯示不同單位之間的轉換關係

In [2]:
! unitconvert energy list -m

Unit table %s can convert between the units:
btu (x * 0.00094781707775)
j (base unit)
wh (x * 0.00027777777778)
hph (x * 3.7250614123e-7)
cal (x * 0.23900573614)
ev (x * 6241506480000000000.0)
kcal (x * 0.00023884589663)


##### 將2500 kcal的能量分別轉換為 cal, j, ev的形式

In [ ]:
! unitconvert energy convert 2500 kcal cal j ev